In [1]:
# Necessary imports
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# Load pickled data
df = pd.read_pickle('../data/kanye_lyrics.pkl')

In [29]:
# Check it out
df.head()

,Album,Song,Year,Run-time,Featured artist(s),Writer(s),Skit,Lyrics,lyric_length,lyrics_cleaned
0,The College Dropout,Intro (Skit),2004,00:19:00,NaN,Kanye West,Y,"Kanye, can I talk to you for a minute? Me and the other faculty members was wonderin' could you do a lil som... Somethin' beautiful, somethin' that the kids is gon' love when they hear it. Tha's gon make them start jumpin' up and down and sharin' candy an' stuff. Think you could probably do somethin' for the kids for graduation to sing?",338,"kanye, can i talk to you for a minute? me and the other faculty members was wonderin' could you do a lil som... somethin' beautiful, somethin' that the kids is gon' love when they hear it. tha's gon make them start jumpin' up and down and sharin' candy an' stuff. think you could probably do somethin' for the kids for graduation to sing?"
1,The College Dropout,We Don't Care,2004,03:59:00,NaN,"Kanye West, Miri Ben-Ari, Ross Vannelli",N,"[Intro]\nOh yeah, I've got the perfect song for the kids to sing\nAnd all my people that's...\n\n[Hook]\nDrug dealin' just to get by\nStack ya' money 'til it get sky high\nWe wasn't s'posed to make it past 25\nJoke's on you, we still alive\nThrow your hands up in the sky and say:\n""We don't care what people say""\n\n[Verse 1]\nIf this is your first time hearin' this\nYou are about to experience somethin' so cold, man\nWe never had nothin' handed, took nothin' for granted\nTook nothin' from no man, man, I'm my own man\nBut as a shorty I looked up to the dopeman\nOnly adult man I knew that wasn't broke, man\nFlickin' Starter coats, man\nMan, you don't know, man\nWe don't care what people say\nThis is for my niggas outside all winter\n'Cause this summer they ain't finna say: ""next summer I'm finna""\nSittin' in the hood like community colleges\nThis dope money here is Lil' Trey's scholarship\n'Cause ain't no tuition for having no ambition\nAnd ain't no loans for sittin' your ass at home\nSo we forced to sell crack, rap, and get a job\nYou gotta do somethin' man, your ass is grown\n\n[Hook]\nDrug dealin' just to get by\nStack ya' money 'til it get sky high\n(Kids, sing! Kids, sing!)\nWe wasn't s'posed to make it past 25\nJoke's on you, we still alive\nThrow your hands up in the sky and say:\n""We don't care what people say""\n\n[Verse 2]\nThe second verse is for my dogs workin' 9 to 5 that still hustle\n'Cause a nigga can't shine off $6.55\nAnd everybody sellin' makeup, Jacob's\nAnd bootlegged tapes just to get they cake up\nWe put shit on layaway, then come back\nWe claim other people kids on our income tax\nWe take that money, cop work, then push packs to get paid\nAnd we don't care what people say\nMomma say she wanna move South\nScratchin' lottery tickets, eyes on a new house\n'Round the same time, Doe ran up in dude house\nCouldn't get a job, so since he couldn't get work\nHe figured he'd take work\nThe drug game bulimic, it's hard to get weight\nA nigga's money is homo, it's hard to get straight\nBut we gon' keep bakin' til the day we get cake\nAnd ""we don't care what people say""\n\n[Hook]\nDrug dealin' just to get by\nStack ya' money 'til it get sky high\n(Kids, sing! Kids, sing!)\nWe wasn't s'posed to make it past 25\nJoke's on you, we still alive\nThrow your hands up in the sky and say:\n""We don't care what people say""\n\n[Verse 3]\nYou know the kids gon' act a fool\nWhen you stop the programs for after school\nAnd they DCFS, some of 'em dyslexic\nThey favorite 50 Cent song ""12 Questions""\nWe scream: ""rocks, blow, weed, park"", see, now we smart\nWe ain't retards, the way teachers thought\nHold up, hold fast, we make more cash\nNow tell my momma I belong in that slow class\nSad enough we on welfare\nThey tryna put me on the school bus with the space for the wheelchair\nI'm tryna get the car with the chromie wheels here\nThey tryna cut our lights out like we don't live here\nLook what was handed us, fathers abandoned us\nWhen we get them hammers, go on, call the ambulance\nSometimes I f

### Get all lyrics in a long single string

In [11]:
lyrics_str = ''
for i, row in df.iterrows():
    lyrics_str += row['lyrics_cleaned'] + ' '

### Prep string in the form of (sequences, following character) pairs to train an LSTM

In [16]:
seq_len = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(lyrics_str) - seq_len, step):
    sentences.append(lyrics_str[i: i + seq_len])
    next_chars.append(lyrics_str[i + seq_len])
print('Number of sequences:', len(sentences))


chars = sorted(list(set(lyrics_str)))
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

Number of sequences: 98438
Unique characters: 69


In [37]:
n_chars = len(lyrics_str)
n_vocab = len(chars)
n_sentences = len(sentences)

### Convert sequences and following characters to vector forms

In [32]:
X = np.zeros((len(sentences), seq_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
 
    y[i, char_indices[next_chars[i]]] = 1
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1

In [33]:
X.shape

(98438, 40, 69)

In [34]:
y.shape

(98438, 69)

In [38]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, n_vocab)))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# define the checkpoint
filepath="kanye_model_6_7_19/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(x, y, epochs=100, batch_size=128, callbacks=callbacks_list)